In [1]:
import numpy as np

class RigidBodyTransformation:
    def __init__(self, P, Q):
        """
        Initialize with two 3D point sets P and Q.

        Parameters:
        P (numpy.ndarray): 3D point set before transformation (n x 3)
        Q (numpy.ndarray): 3D point set after transformation (n x 3)
        """
        self.P = P
        self.Q = Q
        self.centroid_P = np.mean(self.P, axis=0)
        self.centroid_Q = np.mean(self.Q, axis=0)
        self.T = None  # Translation vector
        self.R = None  # Rotation matrix

    def compute_translation(self):
        """
        Compute the translation vector T as the difference between centroids.
        """
        self.T = self.centroid_Q - self.centroid_P

    def compute_rotation(self):
        """
        Compute the rotation matrix R using SVD.
        """
        # Step 2: Align centroids
        P_centered = self.P - np.mean(self.P, axis=0)
        Q_centered = self.Q - np.mean(self.Q, axis=0)

        # Step 3: Compute rotation matrix
        H = P_centered.T @ Q_centered  # Covariance matrix
        U, S, Vt = np.linalg.svd(H)    # SVD decomposition
        self.R = Vt.T @ U.T


    def infer_transformation(self):
        """
        Perform the full inference of translation and rotation.
        """
        self.compute_translation()
        self.compute_rotation()
        return self.T, self.R

    def apply_transformation(self, P):
        """
        Apply the computed transformation to a given 3D point set P.

        Parameters:
        P (numpy.ndarray): 3D point set to transform (n x 3)

        Returns:
        numpy.ndarray: Transformed point set.
        """
        if self.T is None or self.R is None:
            raise ValueError("Transformation not yet computed. Call infer_transformation first.")
        
        # center it according to the centroid of P, then rotate it, then translate it
        return ((P-self.centroid_P) @ self.R.T) + self.centroid_P + self.T


In [2]:
P = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
Q = np.array([[0.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0]])

transformer = RigidBodyTransformation(P, Q)

T, R = transformer.infer_transformation()

In [3]:
T

array([0.33333333, 0.33333333, 0.33333333])

In [4]:
R

array([[-0.33333333,  0.66666667,  0.66666667],
       [ 0.66666667, -0.33333333,  0.66666667],
       [ 0.66666667,  0.66666667, -0.33333333]])

In [5]:
transformed_P = transformer.apply_transformation(P)

In [6]:
transformed_P

array([[ 0.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00, -1.11022302e-16,  1.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00]])